In [44]:
import pandas as pd 
import os 
import sqlite3 
import shutil
import numpy as np
from datetime import datetime
from datetime import date
import datetime
import time
import glob
# adding glob for file pathing
import win32com
import win32com.client as win32


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# connect to the database
production = sqlite3.connect('C:\\python_development\\productivity.db', timeout=220) 

In [46]:
# password stripping function updated for windows 11
# import all files from EPIC into c:\python_development\epic_import with the password Ucr2022!
# citrix link for the 

def Remove_password_xlsx(filename, pw_str):
    try:
        if not filename.startswith('~$'):  # Ignore temporary Excel files
            xcl = win32.Dispatch('Excel.Application')
            wb = xcl.Workbooks.Open(filename, False, False, None, pw_str)
            xcl.DisplayAlerts = False
            wb.SaveAs(filename, None, '', '')  # Removing the password
            wb.Close(True)
            xcl.Quit()
            print(f"Password removed for file: {filename}")
        else:
            print(f"Skipping temporary file: {filename}")
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

files = [file for file in os.listdir('C:\\python_development\\epic_import') if file.endswith('.xlsx')]

for file in files:
    Remove_password_xlsx(os.path.join('C:\\python_development\\epic_import', file), 'Ucr2022!')


Password removed for file: C:\python_development\epic_import\PHSO_NCQA_Quality_All_Advanced_Care_Plan__SA_20240930_0658.xlsx
Password removed for file: C:\python_development\epic_import\UCR_Health_In_Basket_Statistics_for_Recipients_I_Manage_20240930_0656.xlsx
Password removed for file: C:\python_development\epic_import\UC_PHSO_DES_Quality_All_Breast_Cancer_Screening__SA_20240930_0658.xlsx
Password removed for file: C:\python_development\epic_import\UC_PHSO_DES_Quality_All_Cervical_Cancer_Screening__SA_20240930_0658.xlsx
Password removed for file: C:\python_development\epic_import\UC_PHSO_DES_Quality_All_Colorectal_Cancer_Screening__SA_20240930_0658.xlsx
Password removed for file: C:\python_development\epic_import\UC_PHSO_DES_Quality_All_Depression_Screening__SA_20240930_0658.xlsx
Password removed for file: C:\python_development\epic_import\UC_PHSO_NCQA_All_Diabetes_Optimal_Care__SA_20240930_0657.xlsx
Password removed for file: C:\python_development\epic_import\UC_PHSO_NCQA_Quality_Hyp

In [47]:
# loop to import file data to sqlite appending the new reports, adding a report date, and creating a unique report identifier

for file in files:
    if file.__contains__('All_Breast_Cancer_Screening'):
        # if there are errors in the execution, UCSD may have renamed some columns, usecols method is required as columns come and go
        # have to update all file calls to use os.path.join method
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Last Mammo', 'Breast Cancer Measure',
       'Last Primary Care Visit', 'Next PCP Visit', 'Pt Comm Pref',
       'Pt. Portal Status', 'PCP', 'PCP Department', 'Payer', 'PHT Eligible',
       'Engaged with Population Health Team', 'Race', 'Ethnicity', 'ZIP Code',
       'Active Patient HPI Score', 'Active Patient HPI Percentile'], parse_dates=['Last Primary Care Visit'])
        df.columns = df.columns.str.replace(' ','_').str.lower()
        # for example, they changed the column name from payor to payer and now I have to rename it so it matches the existing columm
        df.rename(columns={'payer': 'payor'}, inplace=True)
        # getting the report date from the filename which is automatically generated on export
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        # unique key for each report is the patient mrn with the report date, this ensures unique rows and trending per patient
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_breast', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_breast', production)
        # de-duplication method to ensure unique rows, layered for data integrity
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_breast', production, if_exists='replace', index=False)
    elif file.__contains__('Cervical_Cancer_Screening'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Last Pap Smear Date',
       'Last Pap Smear Department', 'Cervical Cancer Measure',
       'Last Primary Care Visit', 'Next PCP Visit', 'Pt Comm Pref',
       'Pt. Portal Status', 'PCP', 'PCP Department', 'Payer', 'PHT Eligible',
       'Engaged with Population Health Team', 'Race', 'Ethnicity', 'ZIP Code',
       'Active Patient HPI Score', 'Active Patient HPI Percentile'], parse_dates=['Last Pap Smear Date','Last Primary Care Visit'])
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df.rename(columns={'payer': 'payor'}, inplace=True)
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_cervical', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_cervical', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_cervical', production, if_exists='replace', index=False)
    elif file.__contains__('Colorectal_Cancer_Screening'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Last Colonoscopy Dt', 'Last FIT Date',
       'Last FIT', 'Last Cologuard Date', 'Last Cologuard Value',
       'Colorectal HM Due', 'Colorectal Cancer Screening',
       'Last Primary Care Visit', 'Next PCP Visit', 'Pt Comm Pref',
       'Pt. Portal Status', 'PCP', 'Payer', 'PCP Department', 'PHT Eligible',
       'Engaged with Population Health Team', 'Race', 'Ethnicity', 'ZIP Code'], parse_dates=['Last Colonoscopy Dt','Last Primary Care Visit'])
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df.rename(columns={'payer': 'payor'}, inplace=True)
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_colorectal', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_colorectal', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_colorectal', production, if_exists='replace', index=False)
    elif file.__contains__('All_Depression_Screening'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Last PHQ-9 Score', 'Last PHQ-9 Date',
       'MYC PHQ-2 Last Score', 'Last PHQ-2 Score', 'Last PHQ-8 Score',
       'Depression Y/N', 'Depression Screening', 'Last Primary Care Visit',
       'Next PCP Visit', 'Pt Comm Pref', 'Pt. Portal Status', 'PCP',
       'PCP Department', 'Payer', 'PHT Eligible',
       'Engaged with Population Health Team', 'Race', 'Ethnicity', 'ZIP Code',
       'Active Patient HPI Score', 'Active Patient HPI Percentile'], parse_dates=['Last PHQ-9 Date','Last Primary Care Visit'])
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df.rename(columns={'payer': 'payor'}, inplace=True)
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_depression', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_depression', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_depression', production, if_exists='replace', index=False)
    elif file.__contains__('All_Diabetes_Optimal_Care'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Last A1c', 'Last A1c Dt',
       'Last Creatinine', 'Last Creatinine Dt', 'Systolic BP', 'Diastolic BP',
       'Last BP Dt', 'DM A1c >8', 'DM BP >140/90', 'DM Eye Exam',
       'DM Nephropathy', 'DM Optimal Care Measure', 'Last PCP Visit Date',
       'Next PCP Visit', 'PCP', 'PCP Department', 'Pt Comm Pref',
       'Pt. Portal Status', 'Payer', 'Primary Plan', 'PHT Eligible',
       'Engaged with Population Health Team', 'HM Protocol Order Date',
       'Is on QIP Registry?', 'Enrolled in Digital Health',
       'Active Patient HPI Percentile', 'ZIP Code', 'Race', 'Ethnicity'], parse_dates=['Last A1c Dt','Last Creatinine Dt','Last BP Dt','Last PCP Visit Date'])
        # Last PCP Visit Date   Last PCP visit
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df.rename(columns={'payer': 'payor','last_pcp_visit_date':'last_pcp_visit'}, inplace=True)
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_diabetes', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_diabetes', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_diabetes', production, if_exists='replace', index=False)
    elif file.__contains__('Hypertension_All_Patients'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Systolic BP', 'Diastolic BP',
       'Last BP Dt', 'Last BP Department', 'BP Control Measure',
       'On Antihypertensives', 'Last BMI', 'Last Primary Care Visit',
       'Next PCP Visit', 'Pt Comm Pref', 'Pt. Portal Status', 'PCP',
       'PCP Department', 'Payer', 'Primary Plan', 'PHT Eligible',
       'Engaged with Population Health Team', 'Enrolled in Digital Health',
       'Race', 'Ethnicity', 'Active Patient HPI Percentile', 'ZIP Code'],parse_dates=['Last BP Dt','Last Primary Care Visit','Next PCP Visit'])
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df.rename(columns={'payer': 'payor'}, inplace=True)
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_hypertension', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_hypertension', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_hypertension', production, if_exists='replace', index=False)
    elif file.__contains__('Advanced_Care_Plan'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file), usecols=['MRN', 'Patient', 'Age', 'Sex', 'Adv Dir', 'ACP Date', 'ACP Measure',
       'Last Primary Care Visit', 'Next PCP Visit', 'Pt Comm Pref',
       'Pt. Portal Status', 'PCP', 'PCP Department', 'HCC Needs Recertification',
       'Payer', 'PHT Eligible', 'Engaged with Population Health Team', 'Race',
       'Ethnicity', 'ZIP Code'],parse_dates=['ACP Date','Last Primary Care Visit','Next PCP Visit'])
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df.rename(columns={'payer': 'payor','hcc_needs_recertification': 'hcc_needs_refresh'}, inplace=True)
        df['snapshot_dtm'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.mrn.astype(str) + "-" + df.snapshot_dtm.astype(str)
        df.to_sql('src_hedis_advanced_care_plan', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_hedis_advanced_care_plan', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_hedis_advanced_care_plan', production, if_exists='replace', index=False)
    elif file.__contains__('In_Basket_Statistics_for_Recipients_I_Manage'):
        df = pd.read_excel(os.path.join('C:\\python_development\\epic_import', file))
        df.columns = df.columns.str.replace(' ','_').str.lower()
        df['report_date'] = datetime.datetime(int(file[-18:-14]), int(file[-14:-12]), day = int(file[-12:-10]))
        df['import_key'] = df.recipient_name.str[:3] + "-" + df.report_date.astype(str)
        df.to_sql('src_provider_inbox', production, if_exists='append', index=False)
        update = pd.read_sql_query('select * from src_provider_inbox', production)
        update.drop_duplicates(inplace=True)
        update.to_sql('src_provider_inbox', production, if_exists='replace', index=False)
    else:
        print('importing completed')

In [48]:
# main functions for transforming provider in basket report

# import the existing source data which was updated in the cell above
inbox = pd.read_sql_query('select * from src_provider_inbox', production)

# clean up the column headers
inbox.columns = inbox.columns.str.lower().str.replace(' ','_').str.replace('(','').str.replace(')','')

# cleaning up the message types
inbox['message_benchmark'] = inbox.folder_name_benchmark_in_days.str.split(pat='(', expand=True)[1].str.replace(')','').astype('float')
inbox['message_type'] = inbox.folder_name_benchmark_in_days.str.split(pat='(', expand=True)[0]

inbox['message_age_total'] = inbox.average_age_in_days * inbox.total
inbox['message_variance'] = inbox.message_age_total -  (inbox.message_benchmark * inbox.total)

# add in old msg variance
inbox['old_message_age_total'] = inbox.average_age_in_days * inbox.num_old_msgs
inbox['old_message_variance'] = inbox.old_message_age_total -  (inbox.message_benchmark * inbox.num_old_msgs)

# add in high priority msg variance
inbox['high_priority_message_age_total'] = inbox.high_priority_average_age_in_days * inbox.high_priority
inbox['high_priority_message_variance'] = inbox.high_priority_message_age_total -  (inbox.message_benchmark * inbox.high_priority)

inbox['high_priority_ratio'] = inbox.high_priority / inbox.total
inbox['old_high_priority_ratio'] = inbox.num_old_high_priority_msgs / inbox.total

abbr = {'Outside Events ':'Outs Evnt', 'Results ':'Rslt', 'My Open Non-Billable Enc ':'Open N-Bill',
       'Rx Request ':'Rx Rqst', 'Pt Messages ':'Pt Msg', "CC'd Charts ":"CC Chrt",
       'Patient Questionnaires ':'Pt Qst', 'My Open Billable Enc ':'Open Bill',
       'My Incomplete Notes ':'Inc Notes', 'Refill Errors ':'Refill Err', 'Comm Mgr ':'Comm Mgr',
       'Medication Cancellation ':'Med Cx', 'MyChart Notifications ':'MyChart Notif',
       'Pt Reminder ':'Pt Rem', 'Media Manager ':'Media Mgr', 'Staff Message ':'Staff Msg',
       'Patient Call ':'Pt Call', "CC'd Results ":"CC Rslt", 'Cosign - Clinic Orders ':'Clin Ord Cosign',
       'Cosign-Charts ':'Cosign Chart', 'Timeout Msg ':'Timeout Msg', 'My Unsigned Orders ':'Unsign Ord',
       'Letter Drafts ':'Ltr Draft', 'Referral Message ':'Ref Msg', 'Rx Response ':'Rx Resp',
       'Addendum Notification ':'Add Notif', 'UCI Questionnaire Submission ':'UCI Quest Sub',
       'Follow-up Reminder ':'Follow Rem', 'Overdue Rx Request ':'Over Rx Req', 'Outside Messages ':'Outs Msg',
       'Result Notes ': 'Res Notes', 'E-Prescribing Error ': 'E-Rx Err', 'Message Routing ':'Msg Rout',
       'Post Mortem ': 'Post Mortem', 'Cosign Notes ':'Cosign Notes',
       'Hospital Admissions and Discharges ': 'IP Adm DC', 'Letters ':'Letters',
       'Overdue Message ':'Overdue Msg'}
inbox['task_abbr'] = inbox['message_type'].map(abbr)
# update the stating table
inbox.to_sql('stg_provider_inbox', production, if_exists='replace', index=False)

65139

In [49]:
# providers are identified with string matching because EPIC report does not include provider ID
# reporting table also includes logic to identify and label latest report

query = '''
with case_matches as
(
select 
s.*
, case    
        when recipient_name = 'FRANKLIN, DAVID' then 40931
        when recipient_name = 'NYC, MARY' then 306234
        when recipient_name = 'BALLOU, KENNETH' then 70397
        when recipient_name = 'PATEL, ZALAK' then 306662
        when recipient_name = 'HANDLER, STEPHANIE' then 308213
        when recipient_name like '%CAS%' and recipient_name like '%ROB%' then 40929
        when recipient_name like 'CASTA%' then 40929
        when recipient_name = 'FICHTNER, CHRISTOPHER GLENN' then 71263
        when recipient_name = 'YAMAGUCHI, TOSHIA-ANN' then 264165
        when recipient_name = 'SPRING, JACQUELYN' then 290897
        when recipient_name = 'BOUTROS, NADER' then 275871
        when recipient_name = 'GHOSTINE, SAMER' then 40933
        when recipient_name = 'LAKES, KIMBERLEY DAWN' then 42298
        when recipient_name = 'KOKIKIAN, COLLETTE' then 42298
        when recipient_name = 'GOWDA, NANDINI' then 269915
        when recipient_name = 'AL-KHOURY, LAMA' then 50471
        when recipient_name = 'NEVIN, JOHN J' then 275131
        when recipient_name = 'RUSSELL, JEAN' then 275618
        when recipient_name = 'OMI, KAZUO' then 287200
        when recipient_name = 'PATEL, MANDIRA' then 263266
        when recipient_name = 'DEBAY, MARC' then 70133
        when recipient_name = 'NGUYEN, PATRICK' then 290652
        when recipient_name = 'LE, ANDREW' then 290738
        when recipient_name = 'PORTA, LINDSEY' then 290770
        when recipient_name = 'PATEL, CHIRAG' then 305699
        when recipient_name = 'PATEL, AVI' then 306227
        when recipient_name = 'SEBETI, ESMAEIL' then 14145
        when recipient_name = 'INGRAM, MICHAEL' then 62164
        else p.provider_id end as provider_id
from stg_provider_inbox as s
left outer join vw_provider_dimension as p
on s.recipient_name = p.provider_name
)
select
    high_priority_average_age_in_days
    , average_age_in_days
    , folder_name_benchmark_in_days as task_type
    , pd.provider_name as recipient_name
    , high_priority
    , num_old_high_priority_msgs
    , num_old_msgs as old_task_count
    , new as new_tasks
    , num_old_msgs_high_priority
    , total as total_tasks
    , issues
    , last_login_department
    , report_date as report_week
    , case when report_date = (select max(report_date) from stg_provider_inbox) 
        then 'current_report' else 'previous_report' end as report_filter
    , pd.provider_id
    , pd.subdivision_name
    , pd.activity_name
    , pd.primary_specialty as provider_specialty
    , message_benchmark
    , message_type
    , task_abbr
    , high_priority_ratio
    , old_high_priority_ratio
    , message_age_total
    , message_variance
    , old_message_age_total
    , old_message_variance
    , high_priority_message_age_total
    , high_priority_message_variance
from case_matches as s
left outer join vw_provider_dimension as pd
    on s.provider_id = pd.provider_id
'''
df = pd.read_sql_query(query, production)
df.to_csv('C:\\python_development\\report_csv\\provider_inbox_report.csv', index=False)
df.to_sql('rpt_provider_inbox_report', production, if_exists='replace', index=False)

65139

In [50]:
# additional layer of data integrity to identify and label latest report
update = '''
update rpt_provider_inbox_report
set report_filter = 'current_report'
where report_week = (select max(report_week) from rpt_provider_inbox_report) 
'''
production.execute(update)
production.commit()

In [51]:
# write the report to csv, need to update file location
df = pd.read_sql_query('select * from rpt_provider_inbox_report', production)
df.to_csv('C:\\python_development\\report_csv\\provider_inbox_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\provider_inbox_report.txt', index=False, sep='\t')

In [52]:
staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.last_mammo
    , s.breast_cancer_measure
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_breast) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-breast" as quality_model_join
    , 'breast cancer' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_breast as s on p.patient_mrn = s.mrn
where s.mrn is not null 
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_breast', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_breast_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_breast_report.txt', index=False, sep='\t')

In [53]:
# cervical

staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.last_pap_smear_date
    , s.cervical_cancer_measure
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_cervical) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-cervical"  as quality_model_join
    , 'cervical cancer' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_cervical as s on p.patient_mrn = s.mrn
where s.mrn is not null
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_cervical', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_cervical_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_cervical_report.txt', index=False, sep='\t')

In [54]:
# colorectal

staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.last_colonoscopy_dt
    , s.last_fit_date
    , s.last_fit
    , s.last_cologuard_date
    , s.last_cologuard_value
    , s.colorectal_hm_due
    , s.colorectal_cancer_screening
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_colorectal) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-colorectal"  as quality_model_join
    , 'colorectal cancer' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_colorectal as s on p.patient_mrn = s.mrn
where s.mrn is not null
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_colorectal', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_colorectal_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_colorectal_report.txt', index=False, sep='\t')

In [55]:
# src_hedis_depression

staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.[last_phq-9_score]
    , s.[last_phq-9_date]
    , s.[myc_phq-2_last_score]
    , s.[last_phq-2_score]
    , s.[last_phq-8_score]
    , s.[depression_y/n]
    , s.depression_screening
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_depression) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-depression"  as quality_model_join
    , 'depression' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_depression as s on p.patient_mrn = s.mrn
where s.mrn is not null
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_depression', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_depression_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_depression_report.txt', index=False, sep='\t')

In [56]:
# src_hedis_diabetes

staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_pcp_visit as last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.[dm_a1c_>8]
    , s.[dm_bp_>140/90] as dm_blood_pressure
    , s.systolic_bp as dm_systolic
    , s.diastolic_bp as dm_diastolic
    , s.last_bp_dt as dm_last_bp_dt
    , case
        when s.last_a1c like '%' 
        then replace(s.last_a1c, '%','')
        else s.last_a1c
        end as last_a1c
    , s.last_a1c_dt
    , case when s.last_creatinine like "%NOT%" then null else s.last_creatinine end as last_creatinine
    , s.last_creatinine_dt
    , s.dm_eye_exam
    , s.dm_nephropathy
    , s.dm_optimal_care_measure
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_diabetes) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-diabetes"  as quality_model_join
    , 'diabetes' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_diabetes as s on p.patient_mrn = s.mrn
where s.mrn is not null
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_diabetes', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_diabetes_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_diabetes_report.txt', index=False, sep='\t')

In [57]:
# src_hedis_hypertension

staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.systolic_bp
    , s.diastolic_bp
    , s.last_bp_dt
    , s.bp_control_measure
    , s.on_antihypertensives
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_hypertension) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-hypertension"  as quality_model_join
    , 'hypertension' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_hypertension as s on p.patient_mrn = s.mrn
where s.mrn is not null
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_hypertension', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_hypertension_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_hypertension_report.txt', index=False, sep='\t')

In [58]:
# src_hedis_advanced_care_plan

staging = '''

select 
    p.pat_id
    , p.patient_mrn
    , p.patient_name
    , p.current_pcp_id
    , pd.provider_name
    , pd.subdivision_name
    , pd.activity_code
    , pd.subdivision_code
    , pd.epic_specialty
    , s.last_primary_care_visit
    , s.next_pcp_visit
    , s.payor
    , s.adv_dir
    , s.acp_date
    , s.acp_measure
    , case when s.snapshot_dtm = (select max(snapshot_dtm) from src_hedis_advanced_care_plan) then "current" else "historical" end as report_status
    , strftime("%Y-%m-%d", s.snapshot_dtm) as snapshot_dtm
    , cast(p.current_pcp_id as str) || "-" || strftime("%Y-%m-%d", s.snapshot_dtm) || "-acp"  as quality_model_join
    , 'advanced care plan' as quality_population
from dim_patient as p
left outer join vw_provider_dimension as pd on p.current_pcp_id = pd.provider_id
left outer join src_hedis_advanced_care_plan as s on p.patient_mrn = s.mrn
where s.mrn is not null
and p.current_pcp_id <> -1
and pd.provider_name is not null
'''
df = pd.read_sql_query(staging, production)
df.to_sql('stg_hedis_advanced_care_plan', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_acp_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\hedis_acp_report.txt', index=False, sep='\t')

In [59]:
report = '''
with hypertension as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when bp_control_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_hypertension
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
breast as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when breast_cancer_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_breast
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
cervical as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when cervical_cancer_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_cervical
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
colorectal as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when colorectal_cancer_screening = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_colorectal
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
depression as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when depression_screening = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_depression
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
diabetes as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when dm_optimal_care_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_diabetes
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
acp as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when acp_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_advanced_care_plan
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
)
select * from acp
union
select * from diabetes
union
select * from hypertension
union
select * from depression
union 
select * from colorectal
union
select * from cervical
union 
select * from breast
'''

df = pd.read_sql_query(report, production)
df.to_sql('quality_trending_report', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\quality_trending_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\quality_trending_report.txt', index=False, sep='\t')

In [60]:
report = '''
with hypertension as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when bp_control_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_hypertension
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
breast as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when breast_cancer_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_breast
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
cervical as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when cervical_cancer_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_cervical
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
colorectal as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when colorectal_cancer_screening = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_colorectal
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
depression as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when depression_screening = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_depression
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
diabetes as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when dm_optimal_care_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_diabetes
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
),
acp as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, sum(case when acp_measure = "Met" then 1 else 0 end) as numerator
, count(*) as denominator
, quality_model_join
, quality_population
, report_status
from stg_hedis_advanced_care_plan
group by current_pcp_id, provider_name, snapshot_dtm, quality_model_join, quality_population, report_status, subdivision_name
)
select * from acp
union
select * from diabetes
union
select * from hypertension
union
select * from depression
union 
select * from colorectal
union
select * from cervical
union 
select * from breast
'''


df = pd.read_sql_query(report, production)
df.to_csv('C:\\python_development\\report_csv\\quality_non_compliance_current.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\quality_non_compliance_current.txt', index=False, sep='\t')

In [61]:
query = '''
with date_prox as
(
-- joining dim_date for date evaluations in sqlite
select
q.*
-- using monday snapshots to identify the week start 
-- snapshot MWF schedule started couple weeks after first data pull
, case when strftime('%w',snapshot_dtm) = '1' 
    or snapshot_dtm < '2023-09-06'
    then 'week_start' else null end as week_start
, date(month_start,"+1 day","-1 month") as month_start
-- using month start proximity as the method to identify earliest snapshot in the month
-- snapshots are taken on M,W,F schedule and first of the month is not guaranteed to have snapshot
, julianday(snapshot_dtm) - julianday(date(month_start,"+1 day","-1 month")) as ms_prox
-- julianday() is the sqlite version of converting dates into numerical for comparison
, d.calendar_month_of_year as month_num
-- including numerical month to identify quarter
from quality_trending_report as q
left outer join dim_date as d
on q.snapshot_dtm = date(d.date)
),
month_earliest as
(
-- this cte identifies the earliest snapshot date via the month start proximity
-- grouped by month start which will join to back to the source table via the same column
select
month_start
, min(ms_prox) as min_prox
from date_prox
group by month_start
)
select
d.current_pcp_id
, d.provider_name
, d.subdivision_name
, d.activity_code
, d.subdivision_code
, d.epic_specialty
, d.snapshot_dtm
, d.numerator
, d.denominator
, d.quality_model_join
, d.quality_population
, d.report_status
, d.week_start as week_earliest
, case when m.min_prox = d.ms_prox then 'month_start' 
    else null end as month_earliest
, case when month_num in (1,4,7,10) and m.min_prox = d.ms_prox 
    then 'quarter_start' else null 
    end as quarter_earliest
from date_prox as d
left outer join month_earliest as m
on d.month_start = m.month_start
'''
df = pd.read_sql_query(query, production)
df.to_sql('quality_trending_report', production, if_exists='replace', index=False)
df.to_csv('C:\\python_development\\report_csv\\quality_trending_report.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\quality_trending_report.txt', index=False, sep='\t')

In [62]:
# create row level compliance report to identify multiple gap patients
report = '''
with hypertension as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when bp_control_measure = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_hypertension
where report_status = 'current'
),
breast as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when breast_cancer_measure = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_breast
where report_status = 'current'
),
cervical as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when cervical_cancer_measure = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_cervical
where report_status = 'current'
),
colorectal as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when colorectal_cancer_screening = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_colorectal
where report_status = 'current'
),
depression as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when depression_screening = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_depression
where report_status = 'current'
),
diabetes as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when dm_optimal_care_measure = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_diabetes
where report_status = 'current'
),
acp as
(
select
current_pcp_id
, provider_name
, subdivision_name
, activity_code
, subdivision_code
, epic_specialty
, snapshot_dtm
, case when acp_measure = "Met" then 0 else 1 end as non_compliance
, patient_mrn
, patient_name
, quality_model_join
, quality_population
, 'current' as report_status
from stg_hedis_advanced_care_plan
where report_status = 'current'
)
select * from acp
where non_compliance = 1
union
select * from diabetes
where non_compliance = 1
union
select * from hypertension
where non_compliance = 1
union
select * from depression
where non_compliance = 1
union 
select * from colorectal
where non_compliance = 1
union
select * from cervical
where non_compliance = 1
union 
select * from breast
where non_compliance = 1
'''

df = pd.read_sql_query(report, production)
df.to_csv('C:\\python_development\\report_csv\\quality_non_compliance_current.csv', index=False)
df.to_csv('C:\\python_development\\report_csv\\quality_non_compliance_current.txt', index=False, sep='\t')

In [63]:
# export the refreshed aggregate data from disk to vault
shutil.copy2('C:\\python_development\\report_csv\\provider_inbox_report.csv','V:\\Production\\Data_Model\\export_csv\\provider_inbox_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\provider_inbox_report.txt','V:\\Production\\Data_Model\\export_csv\\provider_inbox_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\quality_trending_report.csv','V:\\Production\\Data_Model\\export_csv\\quality_trending_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\quality_trending_report.txt','V:\\Production\\Data_Model\\export_csv\\quality_trending_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_acp_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_acp_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_acp_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_acp_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_hypertension_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_hypertension_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_hypertension_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_hypertension_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_diabetes_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_diabetes_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_diabetes_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_diabetes_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_depression_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_depression_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_depression_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_depression_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_colorectal_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_colorectal_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_colorectal_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_colorectal_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_cervical_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_cervical_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_cervical_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_cervical_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\hedis_breast_report.csv','V:\\Production\\Data_Model\\export_csv\\hedis_breast_report.csv')
shutil.copy2('C:\\python_development\\report_csv\\hedis_breast_report.txt','V:\\Production\\Data_Model\\export_csv\\hedis_breast_report.txt')
shutil.copy2('C:\\python_development\\report_csv\\quality_non_compliance_current.csv','V:\\Production\\Data_Model\\export_csv\\quality_non_compliance_current.csv')
shutil.copy2('C:\\python_development\\report_csv\\quality_non_compliance_current.txt','V:\\Production\\Data_Model\\export_csv\\quality_non_compliance_current.txt')

'V:\\Production\\Data_Model\\export_csv\\quality_non_compliance_current.txt'

In [64]:
# archive the imported files

for file in files:
    shutil.move(os.path.join('C:\\python_development\\epic_import', file) , os.path.join('V:\\Production\\Data_Model\\epic_archive', file))